In [1]:
from keras.preprocessing.sequence import pad_sequences
import read_mimic_flat
import os
from keras.utils.np_utils import to_categorical
import pickle
import numpy as np
import keras
from keras.models import Model
from keras.layers import Dense, Input, Flatten, Convolution1D, AtrousConvolution1D
from keras.layers import GlobalMaxPooling1D, GlobalMaxPooling1D, Lambda, concatenate
from keras import backend as K
from keras.layers.core import Dropout, Reshape, Activation
from keras.layers.pooling import AveragePooling1D
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.optimizers import Adagrad
from collections import defaultdict
from keras.layers.recurrent import LSTM
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.preprocessing import normalize
from keras.optimizers import SGD
from keras.models import Sequential
from keras.objectives import binary_crossentropy, categorical_crossentropy
#tf.python.control_flow_ops = tf
from full_eval import full_evaluate

np.random.seed(1)
tf.set_random_seed(1)

os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


In [2]:
expand = False
shaped = True
add_desc = False
generlize = True

EMBEDDINGS_SIZE = 100 #D in the paper
nb_epochs = 25
batch_size = 32
num_channels = 300 #H in the paper
filter_len = 4
DROPOUT_P = 0.1 #P in the paper
lmbd = 0 #l2 regularization
hidden_size = 400
trainable_embeddings = True
loss_func = binary_crossentropy

In [3]:
desc_dict = {}
for line in open('mimic/2/ICD9_descriptions'):
    line = line.strip().split('\t')
    desc_dict[line[0]] = line[1]
    
def get_data():
    global VOCAB_SIZE, NUM_OF_CLASSES, SEQ_LEN, reader
    def to_categorical(y, nb_classes=None):
        y = np.array(y)
        if not nb_classes:
            nb_classes = np.max(y) + 1
        n = y.shape[0]
        categorical = np.zeros((n, nb_classes))
        for i, sample in enumerate(y):
            for l in sample:
                categorical[i, l] = 1
        return categorical

    reader = read_mimic_flat.mimic_reader(version, expand=expand, shaped=shaped, add_desc=add_desc, generlize=generlize)
    X_train, X_test, y_trains, y_test, tokens_vocab, labels_vocab = reader.read_mimic()

    VOCAB_SIZE = len(tokens_vocab) + 1
    NUM_OF_CLASSES = len(labels_vocab)


    X_train = pad_sequences(X_train, maxlen=None, dtype='int32',
        padding='pre', truncating='pre', value=len(tokens_vocab))
    X_test = pad_sequences(X_test, maxlen=len(X_train[0]), dtype='int32',
        padding='pre', truncating='pre', value=len(tokens_vocab))

    SEQ_LEN = len(X_train[0])

    y_train = to_categorical(y_trains, NUM_OF_CLASSES)
    y_test = to_categorical(y_test, NUM_OF_CLASSES)
    
    return X_train, X_test, y_train, y_test, tokens_vocab, labels_vocab

In [ ]:
def my_init(shape, name=None):
    value = np.random.random(shape)
    value = 2*value-0.5
    value = value*4*np.sqrt(6)/np.sqrt(shape[0]+shape[1])
    return K.variable(value, name=name)

def my_loss(neg_p):
    def f(output, target):
        out = K.binary_crossentropy(output, target)
        target = target*(1.-neg_p)+neg_p
        out = K.dot(out, K.transpose(target))
        return K.mean(out)
    return f

def create_cbow_model():
    np.random.seed(1)
    tf.set_random_seed(1)
    model = Sequential()
    #sents = Input(shape=(SEQ_LEN,), dtype='int32')
    model.add(Embedding(VOCAB_SIZE, EMBEDDINGS_SIZE, input_length=SEQ_LEN, W_regularizer=l2(lmbd)))
    
    model.add(AveragePooling1D(pool_length=SEQ_LEN))
    #model.add(GlobalMaxPooling1D())
    model.add(Flatten())
    model.add(Dropout(DROPOUT_P))
    
    model.add(Dense(NUM_OF_CLASSES, init=my_init, activation=None, W_regularizer = l2(lmbd)))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    model.compile('adam', loss_func)
    #model.compile('adam', my_loss, metrics=['binary_accuracy'])
    
    return model


def create_cnn_model():
    np.random.seed(1)
    tf.set_random_seed(1)
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, EMBEDDINGS_SIZE, input_length=SEQ_LEN, W_regularizer=l2(lmbd)))

    model.add(Convolution1D(num_channels, filter_len, border_mode='valid', activation=None, W_regularizer = l2(lmbd)))
    #model.add(Activation('relu'))
    model.add(Activation('tanh'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(DROPOUT_P))
    
    if hidden_size:
        model.add(Dense(hidden_size, activation=None, W_regularizer = l2(lmbd)))
        model.add(BatchNormalization())
        #model.add(Activation('relu'))
        model.add(Activation('tanh'))
        model.add(Dropout(DROPOUT_P))

    model.add(Dense(NUM_OF_CLASSES, init=my_init, activation=None, W_regularizer = l2(lmbd)))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    #model.compile('adam', 'categorical_crossentropy', metrics=['binary_accuracy'])
    model.compile('adam', loss_func)

    return model

def create_inc_model():
    np.random.seed(1)
    tf.set_random_seed(1)
    
    sents = Input(shape=(SEQ_LEN,), dtype='int32')
    embeded = Embedding(VOCAB_SIZE, EMBEDDINGS_SIZE, input_length=SEQ_LEN, W_regularizer=l2(lmbd))(sents)
    
    inc_m = []
    for i in range(1, 4):
        conv = Convolution1D(num_channels, i, border_mode='valid', activation=None, W_regularizer = l2(lmbd))(embeded)
        conv = Activation('tanh')(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(DROPOUT_P)(conv)
        inc_m.append(conv)
    merged = concatenate(inc_m)
    pred = Dense(NUM_OF_CLASSES, init=my_init, activation=None, W_regularizer = l2(lmbd))(merged)
    pred = BatchNormalization()(pred)
    pred = Activation('sigmoid')(pred)
    
    model = Model(inputs=sents, outputs=pred)
    #model.compile('adam', 'categorical_crossentropy', metrics=['binary_accuracy'])
    model.compile('adam', loss_func)

    return model

def train(model):
    np.random.seed(1)
    tf.set_random_seed(1)

    early_stopping = EarlyStopping(patience=1)
    model.fit(X_train, [y_train], epochs=nb_epochs, batch_size=batch_size, shuffle = True, verbose=1)
    return model

def evaluate(model, name):
    with open('pred/'+name, 'w') as fp:
        preds = model.predict(X_test, batch_size=batch_size)
        for threshold in [0.5]:
            icd_preds = []
            for pred in preds:
                labels = []
                for i, l in enumerate(pred):
                    if l > threshold:
                        labels.append(labels_vocab[i])
                fp.write(','.join(labels)+'\n')
                icd_preds.append(labels)
            #print('threshold', str(threshold))
            evaluate = reader.evaluate(icd_preds)
            for key in evaluate:
                print(key)
                print(evaluate[key])
        
def evaluate2(model, _):
    preds = model.predict(X_test, batch_size=batch_size)
    
    print('regular eval')
    all_preds = []
    for pred in preds:
        labels = set()
        for i, l in enumerate(pred):
            if l > 0.5:
                labels.add(i)
        all_preds.append(labels)
    
    gold_y = []
    for g in y_test:
        labels = set()
        for i, l in enumerate(g):
            if l > 0.5:
                labels.add(i)
        gold_y.append(labels)
            
    tp, fp, fn =0., 0., 0.      
    for pred, gold in zip(all_preds, gold_y):
        tp += len(pred.intersection(gold))
        fp += len(pred-gold)
        fn += len(gold-pred)
    prec = tp/(tp+fp)
    print(prec)
    recal = tp/(tp+fn)
    print(recal)
    f = 2*(prec*recal)/(prec+recal)
    print(f)



In [ ]:
version = '2'

X_train, X_test, y_train, y_test, tokens_vocab, labels_vocab = get_data()

model1 = create_cbow_model()

print('cbow')
model1 = train(model1)
print(full_evaluate(model1.predict(X_test, batch_size=batch_size), y_test))
#evaluate(model1, 'cbow2')
#evaluate2(model1, 'cbow2')
#evaluate3(model1, 'cbow2')

model2 = create_cnn_model()

print('cnn')
model2 = train(model2) 
print(full_evaluate(model2.predict(X_test, batch_size=batch_size), y_test))
#evaluate(model2, 'cnn2')
#evaluate2(model2, 'cnn2')

#model3 = create_inc_model()

#print('inc')
#model3 = train(model3) 
#evaluate(model3, 'inc2')
#evaluate2(model3, 'inc2')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(73651, 100, input_length=8421, embeddings_regularizer=<keras.reg...)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(pool_size=8421)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(948, kernel_initializer=<function ..., activation=None, kernel_regularizer=<keras.reg...)`


cbow
Epoch 1/25
20533/20533 [==============================] - 21s - loss: 0.5593    
Epoch 2/25
20533/20533 [==============================] - 20s - loss: 0.3637    
Epoch 3/25
20533/20533 [==============================] - 20s - loss: 0.2472    
Epoch 4/25
20533/20533 [==============================] - 20s - loss: 0.1755    
Epoch 5/25
20533/20533 [==============================] - 20s - loss: 0.1296    
Epoch 6/25
20533/20533 [==============================] - 20s - loss: 0.0989    
Epoch 7/25
20533/20533 [==============================] - 20s - loss: 0.0778    
Epoch 8/25
20533/20533 [==============================] - 20s - loss: 0.0629    
Epoch 9/25
20533/20533 [==============================] - 20s - loss: 0.0520    
Epoch 10/25
20533/20533 [==============================] - 20s - loss: 0.0440    
Epoch 11/25
20533/20533 [==============================] - 20s - loss: 0.0380    
Epoch 12/25
20533/20533 [==============================] - 20s - loss: 0.0335    
Epoch 13/25
20533/20

In [ ]:
version = '3'

X_train, X_test, y_train, y_test, tokens_vocab, labels_vocab = get_data()

model1 = create_cbow_model()
print('cbow')
model1 = train(model1)
print(full_evaluate(model1.predict(X_test, batch_size=batch_size), y_test))
#
#evaluate(model1, 'cbow3')
#evaluate2(model1, 'cbow3')

model2 = create_cnn_model()
print('cnn')
model2 = train(model2) 
print(full_evaluate(model2.predict(X_test, batch_size=batch_size), y_test))
#evaluate(model2, 'cnn3')
#evaluate2(model2, 'cnn3')


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(121596, 100, input_length=14042, embeddings_regularizer=<keras.reg...)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `AveragePooling1D` call to the Keras 2 API: `AveragePooling1D(pool_size=14042)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1047, kernel_initializer=<function ..., activation=None, kernel_regularizer=<keras.reg...)`


cbow
Epoch 1/1
31904/49857 [==================>...........] - ETA: 25s - loss: 0.5013

In [7]:
sents = Input(shape=(SEQ_LEN,), dtype='int32')
embeded = model2.layers[0](sents)
conv = model2.layers[1](embeded)
just_conv_model = Model(inputs=sents, outputs=conv)
just_conv_model.compile('adam', loss_func)

In [130]:
def analyze_example(x, y , model, just_conv): 
    prediction = model.predict(x)[0]
    predicted = set()
    for i, l in enumerate(prediction):
        if l > 0.5:
            predicted.add(desc_dict[labels_vocab[i]])
    
    gold = set()
    for i, l in enumerate(y):
        if l > 0.5:
            gold.add(desc_dict[labels_vocab[i]])
    
    print('######TP######')
    print(predicted & gold)
    print('######FP######')
    print(predicted - gold)
    print('######FN######')
    print(gold - predicted)
    
    prediction = just_conv.predict(X_train[:1])[0]
    best_found = []
    for channel in range(num_channels):
        best_score = 0
        best_index = 0
        for i, ngram_scores in enumerate(prediction):
            if ngram_scores[channel] > best_score:
                best_score = ngram_scores[channel]
                best_index = i
        
        best_found.append((best_score, best_index, channel))
    
    print('########max words#######')
    for _, best_index, channel in sorted(best_found, reverse=True)[:30]:
        important = ' '.join([tokens_vocab[num] for num in x[0][best_index-5:best_index+5]])
        if important:
            print('channel num:', channel)
            print(important)

In [131]:
analyze_example(X_train[0:1],y_train[0], model2, model)

######TP######
{'Acute renal failure, unspecified', 'Unspecified septicemia', 'Other shock without mention of trauma', 'Congestive heart failure, unspecified', 'Cardiac arrest', 'Cellulitis and abscess of leg, except foot', 'Acute myocardial infarction, subendocardial infarction, initial episode of care'}
######FP######
set()
######FN######
{'Unspecified protein-calorie malnutrition', 'Other primary cardiomyopathies'}
########max words#######
channel num: 194
in date ) . d . alzheimer 's dementia .
channel num: 245
already taking for right lower extremity cellulitis prior to admission
channel num: 19
ddd ; the total time elapsed from the beginning of
channel num: 56
) . d . alzheimer 's dementia . d .
channel num: 160
groin b.i.d . dd . santyl lotion to heels b.i.d
channel num: 178
, he performed poorly on a modified barium swallowing study
channel num: 284
wound cultures . dd . severe peripheral vascular disease ;
channel num: 241
decision was made to not pursue further invasive proce

In [23]:
len(labels_vocab)

5031

In [8]:
generlize

False

In [24]:
version = '3'

X_train, X_test, y_train, y_test, tokens_vocab, labels_vocab = get_data()

In [26]:
len(labels_vocab)

6527

In [30]:
len(tokens_vocab)

288018

In [57]:
shaped = True
generlize = False
version = '3'
X_train, X_test, y_train, y_test, tokens_vocab, labels_vocab = get_data()
len(tokens_vocab)

121595

In [38]:
1-121595/1005489

0.8790687914039835

In [37]:
words = set()
for line in open('mimic/3/MIMIC_ORIG',encoding='utf-8'):
    line = line.split('|')[3]
    words |= set(line.split())
len(words)

1005489

In [42]:
len(labels_vocab)

1047

In [44]:
len(tokens_vocab)

121595

In [56]:
print(' '.join([tokens_vocab[x] for x in X_test[0] if x < len(tokens_vocab)]))

'' admission date : date discharge date : date date of birth : date sex : f service : medicine allergies : patient recorded as having no known allergies to drugs attending :lf chief complaint : transfer from hospital for cardiac cath major surgical or invasive procedure : cardiac cath with stenting endotracheal intubation cardioversion central line placement history of present illness : age_over_90 year old woman with h/o anemia [ * * d - d * * ] angiodysplasia - related gi bleed , h/o colon cancer , cad s/p anterior mi date , resulting in depressed ef ( dd % ) who was transferred from hospital hosp for cardiac catherization . . recent relevant history : pt had an anterior mi on date and was treated medically at nebh . she did not undergo cardiac catherization at that time . tte showed lvef = dd % with severe hypokinesis of the apex infero - apically to antero - apically . there was akinesis of the distal septum , about halfway to the apex , including the apex . there was no ar , d + m

In [58]:
print(' '.join([tokens_vocab[x] for x in X_test[0] if x < len(tokens_vocab)]))

admission date : date discharge date : date date of birth : date sex : f service : medicine allergies : patient recorded as having no known allergies to drugs attending :lf chief complaint : transfer from hospital for cardiac cath major surgical or invasive procedure : cardiac cath with stenting endotracheal intubation cardioversion central line placement history of present illness : age_over_90 year old woman with h/o anemia [ * * d - d * * ] angiodysplasia - related gi bleed , h/o colon cancer , cad s/p anterior mi date , resulting in depressed ef ( dd % ) who was transferred from hospital hosp for cardiac catherization . . recent relevant history : pt had an anterior mi on date and was treated medically at nebh . she did not undergo cardiac catherization at that time . tte showed lvef = dd % with severe hypokinesis of the apex infero - apically to antero - apically . there was akinesis of the distal septum , about halfway to the apex , including the apex . there was no ar , d + mr ,

In [59]:
len(tokens_vocab)

121595

In [15]:
tokens = []
u = set()
for line in open('mimic/3/MIMIC_FILTERED_DSUMS',encoding='utf-8').readlines()[-2285:]:
    tokens.append(len(line.split('|')[1].split()))
    u |= set(line.split('|')[1].split())
print(len(u))
np.mean(tokens)

35280


1893.3973741794312

In [14]:
tokens = []
u = set()
for line in open('mimic/2/MIMIC_FILTERED_DSUMS',encoding='utf-8').readlines()[:-2285]:
    tokens.append(len(line.split('|')[1].split()))
    u |= set(line.split('|')[1].split())
print(len(u))
np.mean(tokens)

69248


1489.1068192888456